In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [2]:
website = 'https://www.programiz.com'
request = requests.get(website)
soup = BeautifulSoup(request.content, 'html.parser')

tab_link_container = soup.find_all('a', class_ = 'highlight-course__col')

In [3]:
all_course_headings = []
all_course_links = []

In [4]:
for i in tab_link_container:
    all_course_headings.append(i.h3.text)
    all_course_links.append(i.get('href'))

In [5]:
all_course_headings

['Python Programming',
 'C Programming',
 'Java Programming',
 'Javascript Programming',
 'C++ Programming',
 'DS & Algorithms',
 'Kotlin Programming',
 'Swift Programming',
 'C# Programming']

In [6]:
all_course_links

['/python-programming',
 '/c-programming',
 '/java-programming',
 '/javascript',
 '/cpp-programming',
 '/dsa',
 '/kotlin-programming',
 '/swift-programming',
 '/csharp-programming']

In [7]:
def get_data(link):
    request = requests.get(link)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

In [24]:
for link_index in range(len(all_course_links)):
    
    #*********************************************************************************************
    course_description = []
    course_headings = []
    list_headings = []
    
    unordered_list_items = []
    ordered_list_items = []
    links_in_page = []
    site_links = []
    c_link = website + str(all_course_links[link_index])
    #*********************************************************************************************
    site_links.append(c_link)
    course_soup = get_data(c_link)
    right_bar = course_soup.find_all('div', class_ = 'right-bar')
    course_container = course_soup.find_all('div', class_= 'container')
    editor_contents = right_bar[0].find_all('div', class_ = 'editor-contents')
    
    # All links in main page
    for links in right_bar[0].find_all('a'):
        link = str(links.get('href'))
        if 'online-compiler' in link:
            continue
        if 'programming' in link:
            if (website + link) not in site_links:
                site_links.append(website + link)
            else:
                print(link)
    
    cnt = 0
    # traversing page wise from the site_links
    for links in range(len(site_links)):
        cnt += 1
        print(cnt, site_links[links], end=" ")
#         if cnt > 4:
#             break
        page_soup = get_data(site_links[links])
        try:
            right_bar = page_soup.find_all('div', class_ = 'right-bar')
            course_container = page_soup.find_all('div', class_= 'container')
            editor_contents = page_soup.find_all('div', class_ = 'editor-contents')
            print('CC', len(course_container))
            k = 0
            if len(right_bar) == 0:
                right_bar = page_soup.find_all('div', class_ = 'col-sm-12 col-reset')
                if len(right_bar) == 0:
                    right_bar = page_soup.find_all('div', class_ = 'col-sm-12')
                    print('lenght ===>>>', len(right_bar))
                    k = 2
                    editor_contents = page_soup.find_all('div', class_ = 'card card--reference')
                else:
                    editor_contents = page_soup.find_all('div', class_ = 'tabbed-content__right')
                    print('length ===>>>', len(right_bar))
        except IndexError:
            pass
        # Programm Description
        description_str = ''
        for desc in course_container:
            description = desc.find_all('div', class_ = 'program-description')
            if len(description) != 0:
                description_str += description[0].text.replace('\n', ' ').strip() + '|||'
        course_description.append(description_str[:-3])


        # All H1 headings
        head_str = ''
        h1 = right_bar[k].find_all('h1')
        if len(h1) == 0:
            head_str = None
        else: 
            for list_head in h1:
                head_str += list_head.text + '$$$'
            head_str = head_str[:-3]
        course_headings.append(head_str)
        
        # All H3 headings
        list_headings_per_page = ''
        head_str = ''
        for list_head in right_bar[k].find_all('h3'):
            head_str += list_head.text + '$$$'
        list_headings_per_page += head_str[:-3] + '|||'

        # All H2 headings
        head_str = ''
        for list_head in right_bar[k].find_all('h2'):
            head_str += list_head.text + '$$$'
        list_headings_per_page += head_str[:-3]
        list_headings.append(list_headings_per_page)

        # getting unordered lists and links from the progress list column as a str
        unordered_list_items_per_page = ''
        links_per_page = ''
        for i in right_bar[k].find_all('div', class_ = 'progress-list'):
            li_str = ''
            for li in i.find_all('li'):
                li_str += li.text + '$$$'
            unordered_list_items_per_page += li_str[:-3] + '|||'
            link_str = ''
            for page_link in i.find_all('a'):
                link_str += page_link.get('href') + '$$$'
            links_per_page += link_str[:-3] + '|||'
        links_in_page.append(links_per_page[:-3])

        # getting unordered lists from the body
        ul_body = ''
        for ul in editor_contents[k].find_all('ul'):
            ul_li = ''
            for li in ul.find_all('li'):
                ul_li += li.text + '$$$'
            ul_body += ul_li[:-3] + '|||'
        if len(unordered_list_items_per_page) > 0:
            unordered_list_items_per_page += unordered_list_items_per_page + ul_body[:-3]
        elif len(unordered_list_items_per_page) == 0:
            unordered_list_items_per_page = None
        else:
            unordered_list_items_per_page = ul_body[:-3]
        unordered_list_items.append(unordered_list_items_per_page)
        
        # getting Ordered Lists
        ol_body = ''
        for ol in editor_contents[k].find_all('ol'):
            ol_li = ''
            for li in ol.find_all('li'):
                li = li.text.replace('\n',' ')
                ol_li += li + '$$$'
            ol_body += ol_li[:-3] + '|||'
        ol_body = ol_body[:-3]
        if len(ol_body) == 0:
            ol_body = None
        else:
            ol_body.replace('\t',' ')
        ordered_list_items.append(ol_body)
        
        
        
        
    #**************************************************************************************************************
    from csv import writer
    # writing the file
    with open('python.csv', 'w', encoding='utf8', newline='') as f:
        the_writer = writer(f)
        header = ['Links','Chapter Headings','Description', 'List Headings', 
                  'Unordered Lists', 'Ordered Lists']
        the_writer.writerow(header)
        for i in range(len(site_links)):
            row = [site_links[i], course_headings[i], course_description[i], list_headings[i], 
                   unordered_list_items[i], ordered_list_items[i]]
            the_writer.writerow(row)

    # reading the file
    file = pd.read_csv('python.csv')
    dataset = pd.DataFrame(file)

    # viewing first 5 rows
    break

/python-programming/first-program
/python-programming/examples
1 https://www.programiz.com/python-programming CC 6
2 https://www.programiz.com/python-programming/first-program CC 5
3 https://www.programiz.com/python-programming/keywords-identifier CC 5
4 https://www.programiz.com/python-programming/statement-indentation-comments CC 5
5 https://www.programiz.com/python-programming/variables-constants-literals CC 5
6 https://www.programiz.com/python-programming/variables-datatypes CC 5
7 https://www.programiz.com/python-programming/type-conversion-and-casting CC 5
8 https://www.programiz.com/python-programming/input-output-import CC 5
9 https://www.programiz.com/python-programming/operators CC 5
10 https://www.programiz.com/python-programming/namespace CC 5
11 https://www.programiz.com/python-programming/if-elif-else CC 5
12 https://www.programiz.com/python-programming/for-loop CC 5
13 https://www.programiz.com/python-programming/while-loop CC 5
14 https://www.programiz.com/python-progra

In [25]:
len(site_links), len(links_in_page), len(course_headings), len(list_headings), len(unordered_list_items), len(ordered_list_items)

(56, 56, 56, 56, 56, 56)

In [10]:
rr = requests.get('https://www.programiz.com/python-programming/guide')
sp = BeautifulSoup(rr.content, 'html.parser')
right_bar = course_soup.find_all('div', class_ = 'right-bar')
course_container = course_soup.find_all('div', class_= 'container')
editor_contents = sp.find_all('div', class_ = 'content')

In [11]:
len(course_container)

6

In [27]:
dataset

,Links,Chapter Headings,Description,List Headings,Unordered Lists,Ordered Lists
0,https://www.programiz.com/python-programming,NaN,Python is a powerful general-purpose programmi...,Introduction$$$Python Flow Control$$$Python Fu...,Getting Started$$$Keywords and Identifiers$$$S...,NaN
1,https://www.programiz.com/python-programming/f...,How to Get Started With Python?,NaN,1. Run Python in Immediate mode$$$2. Run Pytho...,NaN,Download Thonny IDE.$$$Run the installer to in...
2,https://www.programiz.com/python-programming/k...,Python Keywords and Identifiers,NaN,Rules for writing identifiers$$$Things to Reme...,NaN,Identifiers can be a combination of letters in...
3,https://www.programiz.com/python-programming/s...,"Python Statement, Indentation and Comments",NaN,Multi-line statement$$$Multi-line comments$$$D...,NaN,NaN
4,https://www.programiz.com/python-programming/v...,"Python Variables, Constants and Literals",NaN,Example 1: Declaring and assigning value to a ...,NaN,Constant and variable names should have a comb...
5,https://www.programiz.com/python-programming/v...,Python Data Types,NaN,Table of Contents\n \n|||Data types in Py...,NaN,NaN
6,https://www.programiz.com/python-programming/t...,Python Type Conversion and Type Casting,NaN,Example 1: Converting integer to float$$$Examp...,NaN,Implicit Type Conversion$$$Explicit Type Conve...
7,https://www.programiz.com/python-programming/i...,"Python Input, Output and Import",NaN,Table of Contents\n \n|||Video: Python Ta...,NaN,NaN
8,https://www.programiz.com/python-programming/o...,Python Operators,NaN,Example 1: Arithmetic operators in Python$$$Ex...,NaN,NaN
9,https://www.programiz.com/python-programming/n...,Python Namespace and Scope,NaN,Table of Contents\n \n|||What is Name in ...,NaN,Scope of the current function which has local ...
